In [2]:
from dotenv import load_dotenv
import os
import pg8000
import boto3
from pathlib import Path
import pyarrow.parquet as pq
import io
import logging
import json
from botocore.exceptions import ClientError

In [3]:
def create_secret(secret, secret_identifier):
    try:
        print("here")
        secrets_manager = boto3.client('secretsmanager')
        response = secrets_manager.create_secret(
            Name=secret_identifier,
            SecretString=json.dumps(secret)
        )
        #if secret is created send success message
        print('done')
        return "Success"
    except secrets_manager.exceptions.ResourceExistsException:
        #if name is taken return message
        return "Failure! Secret is taken"
    except Exception as e:
        #other error return this
        return {"UNKNOWN ERROR": e}



In [17]:
def secrets_warehouse():

    secret_identifier = 'cred_DW'
    host = 'nc-data-eng-project-dw-prod.chpsczt8h1nu.eu-west-2.rds.amazonaws.com'
    port =  5432
    user = 'project_team_4'
    password = '9Fy8qy8VsqTjuN'
    database =  'postgres'
    schema =  'project_team_4'

    secret = {
        'user': user,
        'password': password,
        'database': database,
        'host':host,
        'port':port,
        'schema':schema

    }

    return secret, secret_identifier

In [18]:
def entry():
    secret, secret_id = secrets_warehouse()   
    return create_secret(secret, secret_id)  

print(entry())


here
Failure! Secret is taken


In [21]:

def pull_secrets(secret_id):     
    secrets_manager = boto3.client('secretsmanager')

    try:               
        response = secrets_manager.get_secret_value(SecretId=secret_id)  

    except ClientError as e:            
        error_code = e.response['Error']['Code']

        print(error_code)
        if error_code == 'ResourceNotFoundException':            
            raise Exception(f'ERROR: name not found') 
        else:           
            raise Exception(f'ERROR : {error_code}')
    else:
        secrets = json.loads(response['SecretString'])
        details = {
        'user': secrets['user'],
        'password': secrets['password'],
        'database': secrets['database'],
        'host':secrets['host'],
        'port':secrets['port'],
        'schema': secrets['schema']
        }
        return details


In [22]:
print(pull_secrets('cred_DW'))

{'user': 'project_team_4', 'password': '9Fy8qy8VsqTjuN', 'database': 'postgres', 'host': 'nc-data-eng-project-dw-prod.chpsczt8h1nu.eu-west-2.rds.amazonaws.com', 'port': 5432, 'schema': 'project_team_4'}


In [23]:
def make_warehouse_connection(secret_id):
    try:
        details = pull_secrets(secret_id)
        API_HOST = details['host']
        API_USER = details['user']
        API_PASS = details["password"]
        API_DBASE = details["database"]
        conn = pg8000.connect(
            host=API_HOST,
            user=API_USER,
            password=API_PASS,
            database=API_DBASE
        )
        return conn
    except Exception as e:
        print(f"Error connecting to the data warehouse: {str(e)}")
        return None

In [25]:
print(make_warehouse_connection('cred_DW'))